In [3]:
%pip install numpy
%pip install pandas
%pip install -U scikit-learn

     -------------------------------------- 14.8/14.8 MB 693.8 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\Yassmin\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


     -------------------------------------- 10.7/10.7 MB 688.9 kB/s eta 0:00:00
     ------------------------------------ 341.8/341.8 KB 758.8 kB/s eta 0:00:00
     ------------------------------------ 502.3/502.3 KB 732.3 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Yassmin\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


     ---------------------------------------- 8.3/8.3 MB 666.2 kB/s eta 0:00:00
     -------------------------------------- 42.5/42.5 MB 508.8 kB/s eta 0:00:00
     ------------------------------------ 298.0/298.0 KB 557.6 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Yassmin\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [1]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split

In [5]:
pip install dash==1.20.0 

     --------------------------------------- 77.6/77.6 KB 74.4 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  ERROR: Error [WinError 225] Operation did not complete successfully because the file contains a virus or potentially unwanted software while executing command python setup.py egg_info
ERROR: Could not install packages due to an OSError: [WinError 225] Operation did not complete successfully because the file contains a virus or potentially unwanted software

You should consider upgrading via the 'c:\Users\Yassmin\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [7]:
%pip install --upgrade pip
%pip install --root-user-action=ignore requests

     ---------------------------------------- 2.1/2.1 MB 254.8 kB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


                                              0.0/62.5 kB ? eta -:--:--
     ------                                   10.2/62.5 kB ? eta -:--:--
     ------------                           20.5/62.5 kB 217.9 kB/s eta 0:00:01
     ------------                           20.5/62.5 kB 217.9 kB/s eta 0:00:01
     ------------------                     30.7/62.5 kB 163.8 kB/s eta 0:00:01
     ------------------                     30.7/62.5 kB 163.8 kB/s eta 0:00:01
     ------------------                     30.7/62.5 kB 163.8 kB/s eta 0:00:01
     ------------------                     30.7/62.5 kB 163.8 kB/s eta 0:00:01
     ------------------                     30.7/62.5 kB 163.8 kB/s eta 0:00:01
     ------------------                     30.7/62.5 kB 163.8 kB/s eta 0:00:01
     ------------------                     30.7/62.5 kB 163.8 kB/s eta 0:00:01
     ------------------                     30.7/62.5 kB 163.8 kB/s eta 0:00:01
     -------------------------               41.0/62.5 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
%pip install simplet5 -q 

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t

In [10]:
df = pd.read_csv("D:\\CS 462 - Natural Language Processing\\Final Project\\archive\\news_summary.csv", encoding='latin-1', usecols=['headlines', 'text'])

In [11]:
df.head()

,headlines,text
0,Daman & Diu revokes mandatory Rakshabandhan in...,The Administration of Union Territory Daman an...
1,Malaika slams user who trolled her for 'divorc...,Malaika Arora slammed an Instagram user who tr...
2,'Virgin' now corrected to 'Unmarried' in IGIMS...,The Indira Gandhi Institute of Medical Science...
3,Aaj aapne pakad liya: LeT man Dujana before be...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Hotel staff to get training to spot signs of s...,Hotels in Maharashtra will train their staff t...


In [12]:
df = df.rename(columns={"headlines":"target_text", "text":"source_text"})
df = df[['source_text', 'target_text']]

In [13]:
df.head()

,source_text,target_text
0,The Administration of Union Territory Daman an...,Daman & Diu revokes mandatory Rakshabandhan in...
1,Malaika Arora slammed an Instagram user who tr...,Malaika slams user who trolled her for 'divorc...
2,The Indira Gandhi Institute of Medical Science...,'Virgin' now corrected to 'Unmarried' in IGIMS...
3,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Aaj aapne pakad liya: LeT man Dujana before be...
4,Hotels in Maharashtra will train their staff t...,Hotel staff to get training to spot signs of s...


In [15]:
df['source_text'] = "summarize: " + df['source_text']
df

,source_text,target_text
0,summarize: The Administration of Union Territo...,Daman & Diu revokes mandatory Rakshabandhan in...
1,summarize: Malaika Arora slammed an Instagram ...,Malaika slams user who trolled her for 'divorc...
2,summarize: The Indira Gandhi Institute of Medi...,'Virgin' now corrected to 'Unmarried' in IGIMS...
3,summarize: Lashkar-e-Taiba's Kashmir commander...,Aaj aapne pakad liya: LeT man Dujana before be...
4,summarize: Hotels in Maharashtra will train th...,Hotel staff to get training to spot signs of s...
...,...,...
4509,summarize: Fruit juice concentrate maker Rasna...,Rasna seeking ?250 cr revenue from snack categ...
4510,summarize: Former Indian cricketer Sachin Tend...,Sachin attends Rajya Sabha after questions on ...
4511,"summarize: Aamir Khan, while talking about rea...",Shouldn't rob their childhood: Aamir on kids r...
4512,summarize: The Maharashtra government has init...,"Asha Bhosle gets ?53,000 power bill for unused..."


In [14]:
train_df, test_df = train_test_split(df, test_size=0.3)
train_df.shape, test_df.shape

((3159, 2), (1355, 2))

In [3]:
from simplet5 import SimpleT5

model = SimpleT5()

c:\Users\Yassmin\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 42


In [ ]:

model.from_pretrained(model_type="t5", model_name="t5-base")

In [ ]:
model.train(train_df=train_df[:5000],
            eval_df=test_df[:100], 
            source_max_token_len=512, 
            target_max_token_len=100, 
            batch_size=8, max_epochs=5, use_gpu=True)

In [30]:
import os
os.chdir(r'/kaggle/working')

In [31]:
%cd /kaggle/working
from IPython.display import FileLinks
FileLinks(r'outputs')

/kaggle/working


outputs/simplet5-epoch-0-train-loss-0.4144-val-loss-1.4778/
  pytorch_model.bin
  tokenizer_config.json
  tokenizer.json
  special_tokens_map.json
  spiece.model
  config.json
outputs/simplet5-epoch-0-train-loss-1.0477-val-loss-1.2687/
  pytorch_model.bin
  tokenizer_config.json
  tokenizer.json
  special_tokens_map.json
  spiece.model
  config.json
outputs/simplet5-epoch-1-train-loss-0.3607-val-loss-1.5967/
  pytorch_model.bin
  tokenizer_config.json
  tokenizer.json
  special_tokens_map.json
  spiece.model
  config.json
outputs/simplet5-epoch-1-train-loss-1.0041-val-loss-1.2814/
  pytorch_model.bin
  tokenizer_config.json
  tokenizer.json
  special_tokens_map.json
  spiece.model
  config.json
outputs/simplet5-epoch-2-train-loss-0.2924-val-loss-1.703/
  pytorch_model.bin
  tokenizer_config.json
  tokenizer.json
  special_tokens_map.json
  spiece.model
  config.json
outputs/simplet5-epoch-2-train-loss-0.7977-val-loss-1.3159/
  pytorch_model.bin
  tokenizer_config.json
  tokenizer.json
  special_tokens_map.json
  spiece.model
  config.json
outputs/simplet5-epoch-3-train-loss-0.2423-val-loss-1.7435/
  pytorch_model.bin
  tokenizer_config.json
  tokenizer.json
  special_tokens_map.json
  spiece.model
  config.json
outputs/simplet5-epoch-3-train-loss-0.6457-val-loss-1.3693/
  pytorch_model.bin
  tokenizer_config.json
  tokenizer.json
  special_tokens_map.json
  spiece.model
  config.json
outputs/simplet5-epoch-4-train-loss-0.2063-val-loss-1.8198/
  pytorch_model.bin
  tokenizer_config.json
  tokenizer.json
  special_tokens_map.json
  spiece.model
  config.json
outputs/simplet5-epoch-4-train-loss-0.5293-val-loss-1.4517/
  pytorch_model.bin
  tokenizer_config.json
  tokenizer.json
  special_tokens_map.json
  spiece.model
  config.json

In [5]:
model.load_model("t5","D:\\CS 462 - Natural Language Processing\\Final Project\\simplet5-epoch-4-train-loss-0.2063-val-loss-1.8198", use_gpu=False)

In [6]:
text_to_summarize="""summarize: Twitter’s interim resident grievance officer for India has stepped down, leaving the micro-blogging site without a grievance official as mandated by the new IT rules to address complaints from Indian subscribers, according to a source.

The source said that Dharmendra Chatur, who was recently appointed as interim resident grievance officer for India by Twitter, has quit from the post.

The social media company’s website no longer displays his name, as required under Information Technology (Intermediary Guidelines and Digital Media Ethics Code) Rules 2021.

Twitter declined to comment on the development.

The development comes at a time when the micro-blogging platform has been engaged in a tussle with the Indian government over the new social media rules. The government has slammed Twitter for deliberate defiance and failure to comply with the country’s new IT rules.
"""
model.predict(text_to_summarize)

["Twitter's interim resident grievance officer steps down"]

In [10]:
text=""" Egypt’s Foreign Minister Sameh Shoukry met on Monday Chadian President Mohamed Idriss Deby, at the beginning of his two-stop tour in Africa.

Ambassador Ahmed Abu Zeid, the spokesperson of the Ministry of Foreign Affairs, said that Shoukry handed over a letter from President Abdel Fattah Al-Sisi to President Deby addressing, on one side, ways to support and strengthen bilateral relations between the two countries in various fields, and on the other side, consultation and coordination on the ongoing developments in Sudan and the role of Chad and Sudan’s other neighbouring countries in supporting ceasefire efforts and dealing with the humanitarian crisis faced by the brotherly Sudanese people as a result of the ongoing fighting.

 During the meeting, the Chadian president expressed his deep appreciation for the initiative of President AL- Sisi to send the Foreign Minister to Chad at this important time when the brotherly Sudan is facing a very serious security and political challenge. 

He expressed Chad’s aspiration to coordinate with Egypt to encourage the Sudanese parties to establish and adhere to a comprehensive ceasefire that allows the delivery of humanitarian aid and relief, as well as to allow for the launch of a constructive dialogue to resolve differences between the Sudanese parties.

 For his part, Minister Shoukry reviewed the efforts and contacts made by Egypt since the beginning of the crisis, at the political level through direct communication with the parties to the conflict to encourage them to cease fire, address the causes of the dispute through dialogue, and at the humanitarian level by emphasizing the priority of injecting blood and sparing the Sudanese people the ravages of that internecine war.

 Shoukry also said that Egypt opened its arms to receive those fleeing from the hell of the military confrontations by the Sudanese brothers in numbers of more than 60,000 people.

He referred to the great humanitarian tragedy caused by this crisis, which is directly affected by the neighboring countries of Sudan, which requires the intensification of consultation and coordination mechanisms. 

 The two sides agreed to communicate closely at various levels during the next stage, and coordinate positions at the regional and international levels, in order to stop the ongoing war and preserve the integrity, unity and sovereignty of Sudan.

In the same context, Saudi Foreign Minister Faisal bin Farhan discussed with the high representative of the EU for Foreign Affairs and security policy Josep Borrell the latest developments in the situation in Sudan.

During a telephone call on Monday, the two sides reviewed the latest developments of the Saudi – American initiative to host representatives of the Sudanese armed forces and the Rapid Support Forces (RSF) in Jeddah, which aims to create the necessary ground for dialogue to reduce the level of tensions there.

According to a Saudi diplomat on Monday, the ceasefire negotiations between the warring parties in Sudan and held in Saudi Arabia have not made “significant progress,” which weakens hopes for a quick cessation of hostilities.

The Saudi diplomat reported that “negotiations have not made much progress so far, “adding that” a permanent ceasefire is not on the table.. Each side believes that it is able to resolve the battle.

The Sudanese army said earlier that its delegation to the negotiations “will only talk about the truce and how to implement it properly to facilitate humanitarian access.

 """
i=0
list_text=[]
while(i<len(text)):
    list_text.append(model.predict(text[i:i+512]))
    i+=512

In [11]:
print(list_text)

[['Shoukry meets Chadian Prez Deby on Monday'], ['Chad has been instrumental in supporting Sudan ceasefire efforts, expressed gratitude to Sisi'], ['Sudanese Minister Shoukry reflects on efforts made by Egypt during crisis'], ['Stop fire, inject blood, save the people: Shoukry'], ['How do we stop the war?'], ['What have been the latest developments in Sudan?'], ['No progress in the ceasefire negotiations, says Sudanese official']]


In [13]:
pred_text = '-\n'.join(str(item) for s in list_text for item in s)

In [14]:
print(pred_text)

Shoukry meets Chadian Prez Deby on Monday-
Chad has been instrumental in supporting Sudan ceasefire efforts, expressed gratitude to Sisi-
Sudanese Minister Shoukry reflects on efforts made by Egypt during crisis-
Stop fire, inject blood, save the people: Shoukry-
How do we stop the war?-
What have been the latest developments in Sudan?-
No progress in the ceasefire negotiations, says Sudanese official


In [15]:
text=""" What does literacy mean? The ability to read and write may come to mind. However, what if I told you it was more complicated than that? Commonly, people have understood literacy as a binary concept—you are either literate or you are not. But it turns out that literacy looks different across the globe. 

Literacy has been deemed one of the strongest predictors of individual success, and it allows people to finish schooling and secure jobs all around the world2. Literacy also has the power to impact individual well-being and increases life satisfaction. 

The meaning of literacy quickly transforms into something more complex when we consider how its meaning changes from one culture to the next. In other words, being literate in one culture does not necessarily mean you are literate in another. 

Why is this? Because different cultures speak different languages, use different writing systems, and hold different values and practices! In order to capture a more well-rounded definition of literacy, anthropologists have begun to examine what literacy looks like in different cultures around the world.

Anthropologists reason that literacy is a way of thinking which is deeply intertwined with both social and cultural practices around the world. In other words, we must study literacy “in context3” and remember that literacy depends on the culture we are examining it in. 

Because of this relationship between culture and language, it is impossible to separate their profound impacts on one another. When it comes to learning how to read and/or write, individuals are learning how to reproduce their own cultural knowledge in a tangible way. Each culture around the world has had a unique history of language and literacy, so in order to understand literacy more thoroughly, we must examine it on a culture-to-culture basis. ‍

In this way, we see that language and literacy have grown to suit the needs of the culture it is from. For example, some phrases do not translate directly from one language into another—this is often the case with humor as well as unique cultural expressions or sayings. Linguistic anthropologists claim that while not all phrases can be seen across all languages, each language still has all the necessary tools to communicate their needs effectively4. How we learn to read and write directly corresponds to the needs of the culture in which these skills were learned.  

For example, the Zafimaniry are a group from Madagascar well known for their wood carving, art, and music. Currently the Zafimaniry population is estimated at about 25,000 all of which is distributed into approximately 100 villages. Anthropologist Maurice Bloch describes how when literacy is being constructed in a region, there are many forces acting all at once; this includes socioeconomics, local environment, cultural values, traditions, theories, and many others. A large part of the Zafimaniry culture includes knowledge of the body: how it grows, when it matures, and understanding that the body is a part of the “living world.” Bloch displays how much of what people learn to read and write about is culturally specific. The Zafimaniry learn about different facets of the body, growth, maturity and more depending on what phase of life they are in5. Blotch’s work with the Zafimaniry people is one example of the ways literacy differs around the world. 

On the other side of Africa, the Mende people of Sierra Leone are known for practicing crop rotation6 and their secret societies7. The Mende people have access to learning both English and Arabic and use literacy in these languages to strengthen the power of their secret societies8. Anthropologists Caroline Bledsoe and Kenneth Robey explore how the Mende people use their knowledge of the Arabic language to facilitate secrecy within the culture. Individuals who are adept with Arabic use the language as a tool to acquire knowledge and exchange confidential information with others. In this context, literacy is a vessel for secrecy and ultimately restricts the exchange of private information. In fact, one of the greatest crimes for the Mende people is sharing the secrets of their tribe with another! Clearly, how the Mende people use literacy is extremely different from how the Zafimaniry people use literacy.‍

As mentioned previously, literacy grants individuals the ability to reproduce cultural beliefs and ideologies and the possibilities for this are endless. Some more obvious examples of this may be writing a book, reading a poem, or singing a song. But, these simple forms of literacy can quickly transform into powerful and world-changing activities such as producing a political campaign, sharing a religious text, or even suggesting a new economic model. 

If people have the ability to read and write about their culture, they also have the power to propose new ideas. Over time, what cultures believe, as well as which topics they consider to be the most relevant change. This allows people to use literacy to read about, write, and create new ideas for the culture to share. Even further, literacy can redefine entire cultural values over time. This includes the ability to reshape traditions, reconstruct values, and even redefine power dynamics. How we choose to teach learners to read and write has an immediate impact on how an entire culture’s values and traditions will grow and change. Reading and writing has a major impact on individual lives and can propel people into success. Small changes over time will ultimately lead to large cultural shifts in the future—and it all starts with literacy! 

Our understanding of literacy is transformed when evaluated with an anthropological perspective. It becomes clear that literacy is not the same all around the world but rather a skillset that varies from culture to culture. Charities such as Reading is Fundamental, Reading Partners, The World Literacy Foundation, as well as many others are doing great work when it comes to increasing literacy rates around the world. But, there is always more that can be done! As literacy rates continue to climb, it is important to study literacy with the consideration that it looks different for everyone and includes much more than the ability to read or write. Literacy allows individuals to replicate, and redefine cultural ideologies and will continue to have an impact on what and how we learn in the generations to come. So next time you hear about literacy, take a moment to ask yourself, how do you use literacy to impact the world around you? """

In [16]:
i=0
list_text=[]
while(i<len(text)):
    list_text.append(model.predict(text[i:i+512]))
    i+=512

pred_text = '-\n'.join(str(item) for s in list_text for item in s)
print(pred_text)


What is literacy?-
How does literacy change from culture to culture?-
What is literacy like in different cultures around the world?-
How do individuals reproduce their own cultural knowledge?-
How we learn to read and write directly corresponds to culture in which we were raised-
What is literacy in Zafimaniry?-
What people learn about literacy is culturally specific: Blotch’s work with Zafimaniry-
How Mende people use Arabic to strengthen secrecy?-
How the Mende people use literacy is very different from the Zafimaniry people-
People can use literacy to create new ideas for cultures-
How we teach learners to read and write has a profound impact on culture today-
It becomes clear that literacy is not the same everywhere in the world-
How do you use literacy to impact the world around you?
